In [10]:
import scipy.io
from pathlib import Path
import numpy as np
import hsmm_mvpy as hmp
from collections import Counter
import mne

data = scipy.io.loadmat(Path("../data/ar/varForBumps135_100.mat"))

## Re-order channels

In [11]:
channel_data = scipy.io.loadmat(Path("../data/ar/chanlocs.mat"))
channels = [channel_data["chanlocs"][0][i][0][0] for i in range(32)]
channels
# Create new channel ordering to adhere as close as possible to SAT 1 ordering
reindex_channels = channels.copy()
reindex_channels[1] = "FP2"
reindex_channels[2] = "FPZ"
reindex_channels[8:13] = channels[13:18]
reindex_channels[13:18] = channels[23:28]
reindex_channels[18:19] = ["O1", "O2"]
reindex_channels[20:25] = ["FC3", "FCZ", "FC4", "FT7", "FT8"]
reindex_channels[25:30] = channels[18:23]
reindex_channels[30:33] = ["trash1", "trash2"]

## Convert matlab data to 4D NumPy Array to prepare for HMP

In [12]:
# Want (participant, epochs, channels, samples) dimension
# Count occurrences of each subject to get number of epochs
counts = Counter(data["subjects"].squeeze())

# Calculate the differences between consecutive elements to find max epoch length
epoch_lengths = data["y"] - data["x"] + 1
n_samples = np.max(epoch_lengths)

participant_id = data["subjects"][0][0]
participant_epochs = np.zeros((counts[participant_id], 32, n_samples))
result = []
epoch_counter = 0
for start, end, cond, participant in zip(
    data["x"], data["y"], data["conds"], data["subjects"]
):
    # All epoch timings
    # NOTE: NOT ZERO-INDEXED, INDEX 1 IS FIRST SAMPLE
    new_participant = participant[0]
    # print(new_participant)
    if new_participant != participant_id:
        participant_id = new_participant
        # Process data and reset lists?
        result.append(participant_epochs)
        participant_epochs = np.zeros((counts[participant_id], 32, n_samples))
        epoch_counter = 0
    # print(start, end, cond, participant)
    # subtract one because epoch indices are 1-indexed and python is 0-indexed
    epoch_data = data["data"][start[0] - 1 : end[0] - 1].reshape((32, -1))
    # Pad every epoch to max samples length
    epoch_data = np.pad(
        epoch_data,
        ((0, 0), (0, n_samples - epoch_data.shape[1])),
        mode="constant",
        constant_values=np.nan,
    )
    participant_epochs[epoch_counter] = epoch_data
    epoch_counter += 1
# Append the last participant's epochs to the result list
result.append(participant_epochs)

Pad every participant + epoch to max epoch participant

In [13]:
max_length = np.max([res.shape[0] for res in result])
padded_array = np.empty((len(result), max_length, 32, n_samples))
for i, res in enumerate(result):
    padded_result = np.pad(
        res,
        ((0, max_length - res.shape[0]), (0, 0), (0, 0)),
        mode="constant",
        constant_values=np.nan,
    )
    padded_array[i] = padded_result

In [14]:
participants = [f"AR_{i}" for i in range(1, 21)]
hmp_data = hmp.utils.hmp_data_format(
    padded_array, sfreq=100, participants=participants, channels=channels
)
# Re-order channels to be as similar as possible to SAT1
hmp_data = hmp_data.reindex(channels=reindex_channels).drop_sel(
    channels=["trash1", "trash2"]
)

hmp_data = hmp_data.assign_attrs(sfreq=100, lowpass=35.0, highpass=1.0)

Add condition and RT coordinates

In [15]:
conditions = ["TargetFan1", "TargetFan2", "FoilFan1", "FoilFan2"]
epoch_counter = 0
participant_id = data["subjects"][0][0] - 1
results_conditions = np.full(
    (len(hmp_data.participant), len(hmp_data.epochs)), fill_value="", dtype=object
)
results_rts = np.full(
    (len(hmp_data.participant), len(hmp_data.epochs)), fill_value=np.nan
)
for cond, length, participant in zip(data["conds"], epoch_lengths, data["subjects"]):
    new_participant = participant[0] - 1
    if new_participant != participant_id:
        participant_id = new_participant
        epoch_counter = 0
    results_conditions[participant_id, epoch_counter] = conditions[cond[0] - 1]
    results_rts[participant_id, epoch_counter] = (
        float(length[0]) / hmp_data.sfreq
    ) * 1000
    epoch_counter += 1

In [16]:
hmp_data = hmp_data.assign_coords(
    {
        "cond": (("participant", "epochs"), results_conditions),
        "RT": (("participant", "epochs"), results_rts),
    }
)

In [17]:
hmp_data.to_netcdf(Path("../data/ar/data.nc"))

Compare to SAT1 HMP data

In [18]:
hmp_data

<xarray.Dataset>
Dimensions:      (participant: 20, epochs: 789, channels: 30, samples: 300)
Coordinates:
  * participant  (participant) <U5 'AR_1' 'AR_2' 'AR_3' ... 'AR_19' 'AR_20'
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 782 783 784 785 786 787 788
  * channels     (channels) <U6 'FP1' 'FP2' 'FPZ' 'F7' ... 'CPZ' 'CP4' 'TP8'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 293 294 295 296 297 298 299
    cond         (participant, epochs) object 'TargetFan1' 'TargetFan1' ... ''
    RT           (participant, epochs) float64 1.36e+03 1.04e+03 ... nan nan
Data variables:
    data         (participant, epochs, channels, samples) float64 10.99 ... nan
Attributes:
    sfreq:     100
    offset:    0
    lowpass:   35.0
    highpass:  1.0

In [246]:
hmp_data_sat1 = hmp.utils.load_data(Path("../data/sat1/data.nc"))
hmp_data_sat1

<xarray.Dataset>
Dimensions:      (epochs: 200, channels: 30, samples: 993, participant: 25)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 986 987 988 989 990 991 992
    stim         (participant, epochs) float64 nan 1.0 1.0 1.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object '' 'resp_left' ... '' 'resp_left'
    RT           (participant, epochs) float64 nan 683.0 ... nan 1.02e+03
    cue          (participant, epochs) object '' 'SP' 'AC' 'SP' ... 'SP' '' 'AC'
    movement     (participant, epochs) object '' 'stim_left' ... '' 'stim_right'
    trigger      (participant, epochs) object '' ... 'AC/stim_right/resp_left'
  * participant  (participant) object '0001' '0002' '0003' ... '0024' '0025'
Data variables:
    data         (participant, epochs, channels, samples) float64 nan ... nan
Attributes:
    sfreq:           500.0
    offset:          0
    lowpass:         35.0
    highpass:        1.0
    lower_limit_RT:  0.2
    upper_limit_RT:  2

Deprecated, exploring the dataset

In [3]:
data["info"]
# Sampling freq: 100Hz
# Bandwidth filter 1-35 Hz
# Trial length [onset, sep]
# Conditions: TargetFan1, TargetFan2, FoilFan1, FoilFan2
# Max length is 300 samples, 3s

array(['varibles for HSMM code from Qiong; 100 Hz sampling freq.; filter 1 to 35 Hz;lenght trials[Onset, sep], conditions: TargetFan1, TagetFan2, FoilFan1, FoilFan2'],
      dtype='<U156')

In [9]:
# Conditions per epoch index
data["conds"].shape

(14496, 1)

In [13]:
data["subjects"].shape

(14496, 1)

In [17]:
data["data"][0]

array([10.993831 ,  8.723188 , 10.027045 ,  7.504407 , 18.959301 ,
       21.560759 , 14.805117 ,  9.172554 ,  4.9811172, 15.159214 ,
       21.402538 , 16.80298  , 12.275539 ,  6.392374 , 11.632956 ,
       17.042414 , 16.531641 ,  9.345386 ,  4.4990344,  8.106483 ,
       13.541809 , 13.008925 ,  8.935929 ,  5.1235404,  7.2147107,
        9.932727 , 12.233317 , 10.30872  ,  7.9629884,  5.5633454,
        4.691024 , 10.233817 ], dtype=float32)

In [44]:
# Every point of data, electrodes?
data["data"][0:137]

array([[10.993831 ,  8.723188 , 10.027045 , ...,  5.5633454,  4.691024 ,
        10.233817 ],
       [13.505087 , 11.234549 , 10.693406 , ...,  9.04013  ,  9.229184 ,
        13.818634 ],
       [13.468838 ,  9.2393055,  9.529629 , ...,  9.169965 ,  9.655204 ,
        14.178063 ],
       ...,
       [ 3.4505954,  3.7852283,  3.0477   , ...,  2.1767552, -1.1543489,
        -2.7878852],
       [ 5.2995434,  6.2415214,  5.163968 , ...,  3.698343 ,  4.100088 ,
         3.7230742],
       [-6.627756 , -6.254589 , -4.2560973, ..., 11.651623 , 13.535622 ,
        17.530716 ]], dtype=float32)

In [43]:
# For every sample, which participant does it belong to
print(data["subjects"].shape)
data["subjects"]

(14496, 1)
